In [10]:
import os
from osgeo import gdal
import numpy as np
import random
  


In [2]:
def readTif(fileName):
    dataset = gdal.Open(fileName)
    if dataset == None:
        print(fileName + "文件无法打开")
    return dataset

In [3]:
def writeTiff(im_data, im_geotrans, im_proj, path):
    if 'int8' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_UInt16
    else:
        datatype = gdal.GDT_Float32
    if len(im_data.shape) == 3:
        im_bands, im_height, im_width = im_data.shape
    elif len(im_data.shape) == 2:
        im_data = np.array([im_data])
        im_bands, im_height, im_width = im_data.shape
    # 创建文件
    driver = gdal.GetDriverByName("GTiff")
    dataset = driver.Create(path, int(im_width), int(
        im_height), int(im_bands), datatype)
    if(dataset != None):
        dataset.SetGeoTransform(im_geotrans)  # 写入仿射变换参数
        dataset.SetProjection(im_proj)  # 写入投影
    for i in range(im_bands):
        dataset.GetRasterBand(i + 1).WriteArray(im_data[i])
    del dataset

In [4]:
def TifCrop(TifPath, SavePath, CropSize, RepetitionRate):
    dataset_img = readTif(TifPath)
    width = dataset_img.RasterXSize
    height = dataset_img.RasterYSize
    
    print(f"Image Size (H, W):{height, width}")
    proj = dataset_img.GetProjection()
    geotrans = dataset_img.GetGeoTransform()
    img = dataset_img.ReadAsArray(0, 0, width, height)  # 获取数据

    #  获取当前文件夹的文件个数len,并以len+1命名即将裁剪得到的图像
#     new_name = len(os.listdir(SavePath)) + 1
    #  裁剪图片,重复率为RepetitionRate
    new_name = 0
    for i in range(int((height - CropSize * RepetitionRate) / (CropSize * (1 - RepetitionRate)))):
        for j in range(int((width - CropSize * RepetitionRate) / (CropSize * (1 - RepetitionRate)))):
            #  如果图像是单波段
            if(len(img.shape) == 2):
                cropped = img[int(i * CropSize * (1 - RepetitionRate)): int(i * CropSize * (1 - RepetitionRate)) + CropSize,
                              int(j * CropSize * (1 - RepetitionRate)): int(j * CropSize * (1 - RepetitionRate)) + CropSize]
            #  如果图像是多波段
            else:
                cropped = img[:,
                              int(i * CropSize * (1 - RepetitionRate)): int(i * CropSize * (1 - RepetitionRate)) + CropSize,
                              int(j * CropSize * (1 - RepetitionRate)): int(j * CropSize * (1 - RepetitionRate)) + CropSize]
            #  写图像
            writeTiff(cropped, geotrans, proj, SavePath + "/%d.tif" % new_name)
            #  文件名 + 1
            new_name = new_name + 1
    #  向前裁剪最后一列
    for i in range(int((height-CropSize*RepetitionRate)/(CropSize*(1-RepetitionRate)))):
        if(len(img.shape) == 2):
            cropped = img[int(i * CropSize * (1 - RepetitionRate)): int(i * CropSize * (1 - RepetitionRate)) + CropSize,
                          (width - CropSize): width]
        else:
            cropped = img[:,
                          int(i * CropSize * (1 - RepetitionRate)): int(i * CropSize * (1 - RepetitionRate)) + CropSize,
                          (width - CropSize): width]
        #  写图像
        writeTiff(cropped, geotrans, proj, SavePath + "/%d.tif" % new_name)
        new_name = new_name + 1
    #  向前裁剪最后一行
    for j in range(int((width - CropSize * RepetitionRate) / (CropSize * (1 - RepetitionRate)))):
        if(len(img.shape) == 2):
            cropped = img[(height - CropSize): height,
                          int(j * CropSize * (1 - RepetitionRate)): int(j * CropSize * (1 - RepetitionRate)) + CropSize]
        else:
            cropped = img[:,
                          (height - CropSize): height,
                          int(j * CropSize * (1 - RepetitionRate)): int(j * CropSize * (1 - RepetitionRate)) + CropSize]
        writeTiff(cropped, geotrans, proj, SavePath + "/%d.tif" % new_name)
        #  文件名 + 1
        new_name = new_name + 1
    #  裁剪右下角
    if(len(img.shape) == 2):
        cropped = img[(height - CropSize): height,
                      (width - CropSize): width]
    else:
        cropped = img[:,
                      (height - CropSize): height,
                      (width - CropSize): width]
    writeTiff(cropped, geotrans, proj, SavePath + "/%d.tif" % new_name)
    new_name = new_name + 1

In [11]:
def RandomCrop(ImagePath, LabelPath, IamgeSavePath, LabelSavePath, CropSize, CutNum):
    dataset_img = readTif(ImagePath)
    width = dataset_img.RasterXSize
    height = dataset_img.RasterYSize
    proj = dataset_img.GetProjection()
    geotrans = dataset_img.GetGeoTransform()
    img = dataset_img.ReadAsArray(0, 0, width, height)  # 获取哟昂数据
    dataset_label = readTif(LabelPath)
    label = dataset_label.ReadAsArray(0, 0, width, height)  # 获取标签数据

    #  获取当前文件夹的文件个数len,并以len+1命名即将裁剪得到的图像
#     fileNum = len(os.listdir(IamgeSavePath))
    fileNum = 0
    new_name = fileNum + 1
    while(new_name < CutNum + fileNum + 1):
        #  生成剪切图像的左上角XY坐标
        UpperLeftX = random.randint(0, height - CropSize)
        UpperLeftY = random.randint(0, width - CropSize)
        if(len(img.shape) == 2):
            imgCrop = img[UpperLeftX: UpperLeftX + CropSize,
                          UpperLeftY: UpperLeftY + CropSize]
        else:
            imgCrop = img[:,
                          UpperLeftX: UpperLeftX + CropSize,
                          UpperLeftY: UpperLeftY + CropSize]
        if(len(label.shape) == 2):
            labelCrop = label[UpperLeftX: UpperLeftX + CropSize,
                              UpperLeftY: UpperLeftY + CropSize]
        else:
            labelCrop = label[:,
                              UpperLeftX: UpperLeftX + CropSize,
                              UpperLeftY: UpperLeftY + CropSize]
        writeTiff(imgCrop, geotrans, proj,
                  IamgeSavePath + "/%d.tif" % new_name)
        writeTiff(labelCrop, geotrans, proj,
                  LabelSavePath + "/%d.tif" % new_name)
        new_name = new_name + 1

## Greate Sample Images

In [7]:
#  将影像 A B C 裁剪为重复率为0.x的 H,W 的数据集
'''
滑动窗口裁剪函数
TifPath 影像路径
SavePath 裁剪后保存目录
CropSize 裁剪尺寸
RepetitionRate 重复率
'''
TifPath_PAN =  "Data/GISTIF/RAW/PAN/TestCase6_013282321020_0117Oct14SaiKung013282321020_01_P001_PAN_FULL.tif"
SavePath_PAN = "Data/GISTIF/Processed/PAN"

TifPath_MUL =  "Data/GISTIF/RAW/MUL/TestCase6_013282321020_0117Oct14SaiKung013282321020_01_P001_MUL_FULL.tif"
SavePath_MUL = "Data/GISTIF/Processed/MUL"


TifPath_Label =  "Data/GISTIF/RAW/MASK/TestCase6_Mask.tif"
SavePath_Label = "Data/GISTIF/Processed/MASK"

CropSize = 512 # 128
RepetitionRate = 0

TifCrop(TifPath_PAN, SavePath_PAN, CropSize, RepetitionRate)
TifCrop(TifPath_MUL, SavePath_MUL, int(CropSize/4), RepetitionRate)
TifCrop(TifPath_Label, SavePath_Label, CropSize, RepetitionRate)

Image Size (H, W):(3740, 4312)
Image Size (H, W):(935, 1078)
Image Size (H, W):(3740, 4312)


In [9]:
RGB_Path = "Data/Aerial/RGB.png"
RGB_SavePath = "Data/Aerial/RGB"

GT_Path = "Data/Aerial/GT.png"
GT_SavePath = "Data/Aerial/GT"

CropSize = 200 # 128
RepetitionRate = 0

TifCrop(RGB_Path, RGB_SavePath, CropSize, RepetitionRate)
TifCrop(GT_Path, GT_SavePath, CropSize, RepetitionRate)

Image Size (H, W):(1000, 1000)
Image Size (H, W):(1000, 1000)


## Greate Random Samples

In [ ]:
#  裁剪得到300张256*256大小的训练集
RandomCrop(r"Data\data2\tif\data2.tif",
           r"Data\data2\label\label.tif",
           r"Data\train\image1",
           r"Data\train\label1",
           256, 300)
